# Data Collection
---

In [7]:
# import necessary functions

import numpy as np
from numpy import datetime64 as dt64, timedelta64 as td64
import pickle

import sys, os
import shutil
import xarray as xr
import pandas as pd
from matplotlib import pyplot as plt
import cmocean.cm as cmo

from netCDF4 import Dataset, num2date, date2num 
from datetime import datetime, timedelta

from yodapy.datasources import OOI
ooi = OOI(cloud_source=True)

# Local application/library specific imports
#from golive_library import GoliveLibrary as g

from yodapy.utils.creds import set_credentials_file

Set up output and data directories - but if I'm using a ton of data, do I REALLY want to be saving all of it?

In [5]:
# local imports
import my_module as mymod
from importlib import reload
reload(mymod)

<module 'my_module' from '/Users/derya/Documents/GitHub/coastal-upwelling/my_module.py'>

In [6]:
# make sure the output directory exists
this_dir = os.path.abspath('.').split('/')[-1]
this_parent = os.path.abspath('.').split('/')[-2]
out_dir = '../../' + this_parent + '/' + this_dir + '_output'
data_dir = '../../' + this_parent + '/' + this_dir + '_data/canon/'
print('Creating ' + out_dir + ', if needed')
mymod.make_dir(out_dir)
print('Creating ' + data_dir + ', if needed')
mymod.make_dir(data_dir)

Creating ../../GitHub/coastal-upwelling_output, if needed
Creating ../../GitHub/coastal-upwelling_data/canon/, if needed


eh

In [25]:
ooi.sites

site_rd                     array_name  \
0   CE01ISSM              Coastal Endurance   
1   CE01ISSP              Coastal Endurance   
2   CE02SHBP              Coastal Endurance   
3   CE02SHSM              Coastal Endurance   
4   CE02SHSP              Coastal Endurance   
5   CE04OSBP              Coastal Endurance   
6   CE04OSPD              Coastal Endurance   
7   CE04OSPS              Coastal Endurance   
8   CE04OSSM              Coastal Endurance   
10  CE06ISSM              Coastal Endurance   
11  CE06ISSP              Coastal Endurance   
12  CE07SHSM              Coastal Endurance   
13  CE07SHSP              Coastal Endurance   
14  CE09OSPM              Coastal Endurance   
15  CE09OSSM              Coastal Endurance   
16  CP01CNPM                Coastal Pioneer   
17  CP01CNSM                Coastal Pioneer   
18  CP01CNSP                Coastal Pioneer   
19  CP02PMCI                Coastal Pioneer   
20  CP02PMCO                Coastal Pioneer   
21  CP02PMUI                Coastal Pioneer   
22  CP02PMUO                Coastal Pioneer   
23  CP03ISPM                Coastal Pioneer   
24  CP03ISSM                Coastal Pioneer   
25  CP03ISSP                Coastal Pioneer   
26  CP04OSPM                Coastal Pioneer   
27  CP04OSSM                Coastal Pioneer   
29  GA01SUMO         Global Argentine Basin   
30  GA02HYPM         Global Argentine Basin   
31  GA03FLMA         Global Argentine Basin   
32  GA03FLMB         Global Argentine Basin   
34  GI01SUMO            Global Irminger Sea   
35  GI02HYPM            Global Irminger Sea   
36  GI03FLMA            Global Irminger Sea   
37  GI03FLMB            Global Irminger Sea   
39  GP02HYPM            Global Station Papa   
40  GP03FLMA            Global Station Papa   
41  GP03FLMB            Global Station Papa   
43  GS01SUMO          Global Southern Ocean   
44  GS02HYPM          Global Southern Ocean   
45  GS03FLMA          Global Southern Ocean   
46  GS03FLMB          Global Southern Ocean   
48  RS01SBPD      Cabled Continental Margin   
49  RS01SBPS      Cabled Continental Margin   
50  RS01SHDR      Cabled Continental Margin   
51  RS01SLBS      Cabled Continental Margin   
52  RS01SUM1      Cabled Continental Margin   
53  RS01SUM2      Cabled Continental Margin   
54  RS03ASHS          Cabled Axial Seamount   
55  RS03AXBS          Cabled Axial Seamount   
56  RS03AXPD          Cabled Axial Seamount   
57  RS03AXPS          Cabled Axial Seamount   
58  RS03CCAL          Cabled Axial Seamount   
59  RS03ECAL          Cabled Axial Seamount   
60  RS03INT1          Cabled Axial Seamount   
61  RS03INT2          Cabled Axial Seamount   
62  RS05MDPL  Cabled Juan de Fuca Mid-Plate   

                                            site_name  \
0                      Oregon Inshore Surface Mooring   
1    Oregon Inshore Surface Piercing Profiler Mooring   
2      Oregon Shelf Cabled Benthic Experiment Package   
3                        Oregon Shelf Surface Mooring   
4      Oregon Shelf Surface Piercing Profiler Mooring   
5   Oregon Offshore Cabled Benthic Experiment Package   
6        Oregon Offshore Cabled Deep Profiler Mooring   
7     Oregon Offshore Cabled Shallow Profiler Mooring   
8                     Oregon Offshore Surface Mooring   
10                 Washington Inshore Surface Mooring   
11  Washington Inshore Surface Piercing Profiler M...   
12                   Washington Shelf Surface Mooring   
13  Washington Shelf Surface Piercing Profiler Moo...   
14               Washington Offshore Profiler Mooring   
15                Washington Offshore Surface Mooring   
16                           Central Profiler Mooring   
17                            Central Surface Mooring   
18          Central Surface Piercing Profiler Mooring   
19                   Central Inshore Profiler Mooring   
20                  Central Offshore Profiler Mooring   
21                  Upstream Inshore Profiler Mooring   
22                 Upstr

In [9]:
ooi.instruments

array_name  \
0     Cabled Axial Seamount (Cabled Array)   
1     Cabled Axial Seamount (Cabled Array)   
2     Cabled Axial Seamount (Cabled Array)   
3     Cabled Axial Seamount (Cabled Array)   
4     Cabled Axial Seamount (Cabled Array)   
...                                    ...   
5272                     Coastal Endurance   
5273                     Coastal Endurance   
5274                     Coastal Endurance   
5275                     Coastal Endurance   
5276                     Coastal Endurance   

                                              site_name  \
0                                      ASHES Vent Field   
1                                      ASHES Vent Field   
2                                      ASHES Vent Field   
3                                      ASHES Vent Field   
4                                      ASHES Vent Field   
...                                                 ...   
5272  Washington Shelf Surface Piercing Profiler Moo...   
5273  Washington Shelf Surface Piercing Profiler Moo...   
5274  Washington Shelf Surface Piercing Profiler Moo...   
5275  Washington Shelf Surface Piercing Profiler Moo...   
5276  Washington Shelf Surface Piercing Profiler Moo...   

            infrastructure_name              instrument_name   stream_method  \
0     Medium-Power JBox (MJ03B)     Bottom Pressure and Tilt        streamed   
1     Medium-Power JBox (MJ03B)     Bottom Pressure and Tilt        streamed   
2     Medium-Power JBox (MJ03B)     Bottom Pressure and Tilt        streamed   
3     Medium-Power JBox (MJ03B)     Bottom Pressure and Tilt        streamed   
4     Medium-Power JBox (MJ03B)     Bottom Pressure and Tilt        streamed   
...                         ...                          ...             ...   
5272  Surface Piercing Profiler  Single Point Velocity Meter  recovered_cspp   
5273  Surface Piercing Profiler          Spectral Irradiance  recovered_cspp   
5274  Surface Piercing Profiler          Spectral Irradiance  recovered_cspp   
5275  Surface Piercing Profiler            Spectrophotometer  recovered_cspp   
5276  Surface Piercing Profiler            Spectrophotometer  recovered_cspp   

                                stream_rd  
0                       botpt_nano_sample  
1                       botpt_lily_sample  
2                  botpt_nano_sample_24hr  
3                       botpt_heat_sample  
4                       botpt_iris_sample  
...                                   ...  
5272      velpt_j_cspp_metadata_recovered  
5273  spkir_abj_cspp_instrument_recovered  
5274    spkir_abj_cspp_metadata_recovered  
5275     optaa_dj_cspp_metadata_recovered  
5276   optaa_dj_cspp_instrument_recovered  

[5277 rows x 6 columns]

THOUSANDS of instruments!

In [26]:
# Narrow selection by putting desired site reference designator into search
# surface mooring
ooi.search('CE04OSSM')

<Data Source: OOI (3 Instrument Streams)>

In [27]:
ooi.instruments

array_name                        site_name  \
0  Coastal Endurance  Oregon Offshore Surface Mooring   
1  Coastal Endurance  Oregon Offshore Surface Mooring   
2  Coastal Endurance  Oregon Offshore Surface Mooring   

             infrastructure_name           instrument_name stream_method  \
0  Near Surface Instrument Frame  3-Wavelength Fluorometer   telemetered   
1  Near Surface Instrument Frame                       CTD   telemetered   
2                   Surface Buoy              pCO2 Air-Sea   telemetered   

                    stream_rd  
0                flort_sample  
1   ctdbp_cdef_dcl_instrument  
2  pco2a_a_dcl_instrument_air

In [28]:
# Narrow selection by putting desired site reference designator into search
# shallow profiler
ooi.search('CE04OSPD')

<Data Source: OOI (4 Instrument Streams)>

Check the instruments now

In [29]:
ooi.instruments

array_name                                     site_name  \
0  Coastal Endurance  Oregon Offshore Cabled Deep Profiler Mooring   
1  Coastal Endurance  Oregon Offshore Cabled Deep Profiler Mooring   
2  Coastal Endurance  Oregon Offshore Cabled Deep Profiler Mooring   
3  Coastal Endurance  Oregon Offshore Cabled Deep Profiler Mooring   

               infrastructure_name           instrument_name   stream_method  \
0  Wire-Following Profiler (DP01B)  2-Wavelength Fluorometer  recovered_inst   
1  Wire-Following Profiler (DP01B)          CDOM Fluorometer  recovered_inst   
2  Wire-Following Profiler (DP01B)                       CTD  recovered_inst   
3  Wire-Following Profiler (DP01B)          Dissolved Oxygen  recovered_inst   

                           stream_rd  
0  dpc_flnturtd_instrument_recovered  
1   dpc_flcdrtd_instrument_recovered  
2       dpc_ctd_instrument_recovered  
3    dpc_optode_instrument_recovered

We want CTD

In [30]:
# Narrow selection by putting desired site reference designator into search
# 200m platform
ooi.search('CE04OSPS')

<Data Source: OOI (11 Instrument Streams)>

Check the instruments now

In [31]:
ooi.instruments

array_name                                        site_name  \
0   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
1   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
2   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
3   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
4   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
5   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
6   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
7   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
8   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
9   Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   
10  Coastal Endurance  Oregon Offshore Cabled Shallow Profiler Mooring   

         infrastructure_name                         instrument_name  \
0      200m Platform (PC01B)            Bio-acoustic Sonar (Coastal)   
1      200m Platform (PC01B)                                     CTD   
2      200m Platform (PC01B)                        Dissolved Oxygen   
3      200m Platform (PC01B)                             Seawater pH   
4      200m Platform (PC01B)                              pCO2 Water   
5   Shallow Profiler (SF01B)                3-Wavelength Fluorometer   
6   Shallow Profiler (SF01B)                                     CTD   
7   Shallow Profiler (SF01B)                                 Nitrate   
8   Shallow Profiler (SF01B)  Photosynthetically Available Radiation   
9   Shallow Profiler (SF01B)                             Seawater pH   
10  Shallow Profiler (SF01B)                              pCO2 Water   

   stream_method                 stream_rd  
0       streamed       zplsc_echogram_data  
1       streamed       ctdpf_optode_sample  
2       streamed          do_stable_sample  
3       streamed         phsen_data_record  
4       streamed  pco2w_a_sami_data_record  
5       streamed       flort_d_data_record  
6       streamed        ctdpf_sbe43_sample  
7       streamed            nutnr_a_sample  
8       streamed           parad_sa_sample  
9       streamed         phsen_data_record  
10      streamed  pco2w_a_sami_data_record